# create_sql_query_chain
create_sql_query_chain,SQL查询链,是创建生成SQL查询的链,用于将`自然语言`转换成`数据库的SQL查询`

## 举例1:

In [6]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = "8888.216"
db_host = "localhost"
db_database = "db"
db_port = "3306"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库: ",db.dialect)
print("获取数据库中的表: ",db.get_usable_table_names())

db.run("select count(*) from books")

操作的是哪种数据库:  mysql
获取数据库中的表:  ['books', 'todos', 'user']


'[(1,)]'

## 举例2: chain的使用

In [8]:
import os
import dotenv
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db_user = "root"
db_password = "8888.216"
db_host = "localhost"
db_database = "db"
db_port = "3306"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

chain = create_sql_query_chain(chat_model,db)
response =  chain.invoke({"question":"数据库表books一共有多少数据？"})
print(response)



SELECT COUNT(`id`) as `book_count` FROM `books`;


# create_stuff_documents_chain(了解)

create_stuff_documents_chain用于将`多个文档内容`合并成`单个长文本`的链式工具，并一次性传递给LLM处理（而不是分多次处理）。
适合场景：
- 保持上下文完整，适合需要全局理解所有文档内容的任务（如总结、问答）
- 适合处理 少量/中等长度文档 的场景。

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
根据以下文档内容回答问题：
{docs}

问题：香蕉是什么颜色的？
""")


dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 创建链
chain = create_stuff_documents_chain(chat_model, prompt, document_variable_name="docs")

# 文档输入（修正格式和内容）
docs = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一。"
    ),
    Document(
        page_content="香蕉是黄色的水果，通常在成熟时呈现明亮的黄色。未成熟时可能是绿色的，白色可能是误解。"
    ),
    Document(
        page_content="蓝莓是蓝色的浆果，含有丰富的抗氧化物质。"
    )
]

# 执行查询
result = chain.invoke({"docs": docs})
print(result)

香蕉通常在成熟时呈现明亮的黄色，未成熟时可能是绿色的。 
